### Exercice 1

#### 2. Conversion et Analyse du Stockage


In [ ]:
# pandas est la librairie permettant la manipulation de la donnée, notamment avec la structure tabulaire "Dataframe"
import pandas as pd

In [ ]:
# import du fichier csv, la colonne 24 est typé comme un string, car csv non typé et message d'alerte sur cette colonne
df_csv = pd.read_csv(r'C:\Users\Mourad\Desktop\Autoentreprise\EPSI - 2025 2026\Cours\TD1\flight_data_2024.csv', sep = ',',dtype={24: str}) 

In [ ]:
# lecture des 10 premières lignes du Dataframe
df_csv.head(10)

,year,month,day_of_month,day_of_week,fl_date,op_unique_carrier,op_carrier_fl_num,origin,origin_city_name,origin_state_nm,...,diverted,crs_elapsed_time,actual_elapsed_time,air_time,distance,carrier_delay,weather_delay,nas_delay,security_delay,late_aircraft_delay
0,2024,1,1,1,2024-01-01,9E,4814.0,JFK,"New York, NY",New York,...,0,136.0,122.0,84.0,509.0,0,0,0,0,0
1,2024,1,1,1,2024-01-01,9E,4815.0,MSP,"Minneapolis, MN",Minnesota,...,0,130.0,114.0,88.0,622.0,0,0,0,0,0
2,2024,1,1,1,2024-01-01,9E,4817.0,JFK,"New York, NY",New York,...,0,106.0,90.0,61.0,288.0,0,0,0,0,0
3,2024,1,1,1,2024-01-01,9E,4817.0,RIC,"Richmond, VA",Virginia,...,0,111.0,76.0,51.0,288.0,0,0,0,0,0
4,2024,1,1,1,2024-01-01,9E,4818.0,DTW,"Detroit, MI",Michigan,...,0,79.0,70.0,45.0,237.0,0,0,0,0,0
5,2024,1,1,1,2024-01-01,9E,4822.0,JAX,"Jacksonville, FL",Florida,...,0,137.0,120.0,102.0,833.0,0,0,0,0,0
6,2024,1,1,1,2024-01-01,9E,4822.0,LGA,"New York, NY",New York,...,0,169.0,164.0,125.0,833.0,0,0,0,0,0
7,2024,1,1,1,2024-01-01,9E,4823.0,CHS,"Charleston, SC",South Carolina,...,0,118.0,99.0,86.0,641.0,0,0,0,0,0
8,2024,1,1,1,2024-01-01,9E,4823.0,LGA,"New York, NY",New York,...,0,149.0,123.0,101.0,641.0,0,0,0,0,0
9,2024,1,1,1,2024-01-01,9E,4828.0,ITH,"Ithaca/Cortland, NY",New York,...,0,79.0,67.0,43.0,189.0,0,0,0,0,0


In [ ]:
# Ecriture de trois fichiers parquet (commenté car exécuté seulement la première fois)
"""
df_csv.to_parquet('parquet_native_compression.parquet')
df_csv.to_parquet('parquet_gzip_compression.parquet', compression='gzip')
df_csv.to_parquet('parquet_zstd_compression.parquet', compression='zstd')
"""

In [ ]:
# os est une librairie permettant d'intéragir avec notre système d'exploitation, comme par exemple pour lire la métadonnée de fichiers

import os

path = os.getcwd()
files = list(os.listdir(path))

dict_files = {}

for file in files: 
    file_stat = os.stat(file)
    dict_files[file] = file_stat

for item in dict_files :
    if "parquet" in item : 
      print("File: {:50} Size: {} Mo".format(item,int(dict_files[item].st_size / 1024**2)))


File: clients_data.parquet                               Size: 0 Mo
File: parquet_gzip_compression.parquet                   Size: 229 Mo
File: parquet_native_compression.parquet                 Size: 423 Mo
File: parquet_zstd_compression.parquet                   Size: 207 Mo


Nous observons la diminution significative de la taille du fichier stocké.

Trois méthodes de compression, snappy est le plus rapide pour compresser / décompresser, zstd le compromis, et gzip est lent, mais compresse plus (bon pour archives par exemple).

#### Analyse des Temps de Traitement

In [ ]:
# Lecture du fichier parquet avec compression native, à savoir snappy
df_snappy = pd.read_parquet('parquet_native_compression.parquet')

In [3]:
df_zstd = pd.read_parquet('parquet_zstd_compression.parquet')

In [4]:
df_gzip = pd.read_parquet('parquet_gzip_compression.parquet')

##### Impact mémoire

In [ ]:
# création d'une fonction pour analyser la RAM utilisée par chaque Dataframe

def analyze_memory_usage(df):
    # Calculate memory usage for each column
    memory_usage = df.memory_usage(deep=True)
    memory_usage_df = pd.DataFrame({
        'Column': df.columns,
        'Type': df.dtypes,
        'Memory (MB)': memory_usage[1:] / 1024 / 1024
    })
    memory_usage_df = memory_usage_df.sort_values('Memory (MB)', ascending=False)
    return f"Total memory usage : {df.memory_usage(deep=True).sum() / 1024 / 1024:.2f} MB"

dataframes = {
    "df_csv": df_csv,
    "df_snappy": df_snappy,
    "df_zstd": df_zstd,
    "df_gzip": df_gzip
}

for name, df in dataframes.items():
    print(f"{name} : {analyze_memory_usage(df)}")


df_csv : Total memory usage : 4676.93 MB
df_snappy : Total memory usage : 4623.66 MB
df_zstd : Total memory usage : 4623.66 MB
df_gzip : Total memory usage : 4623.66 MB


'\ngénéralement, parquet utilise moins de mémoire. Ici il a été généré à partir du csv, sans former typer les données. \ncsv par défaut va utiliser des types de colonnes "riche" (float64 par exemple ou encore object), ce qui prend de la mémoire\ncomme il n\'y a pas eu d\'optimisations, alors peu d\'écarts ici pour la partie mémoire.\n'

Généralement parquet utilise moins de mémoire. 
Ici il a été généré à partir du csv, sans former typer les données csv par défaut va utiliser des types de colonnes "riche" (float64 par exemple ou encore object), ce qui prend de la mémoire.
Comme il n'y a pas eu d'optimisations, alors peu d'écarts ici pour la partie mémoire.

In [ ]:
import time


# Création d'une fonction permettant de mesurer le temps d'éxécution d'une autre fonction, qui est exécuté sur un Dataframe entrée en argumant de cette première fonction

def mesure_temps_tache_dataframe(df,task_function) :
    start_time = time.time()
    result = task_function(df)
    temps_passe = time.time() - start_time
    return f"temps passé sur la fonction {task_function.__name__}: {temps_passe:.2f} s"

In [ ]:
# Création de trois fonctions faisant des actions sur les Dataframes (agrégations, métadonnées, ...) afin de pouvoir comparer avec la fonction ci-dessus, le temps que prennent chacune de ces fonctions, pour chaque Dataframe

def decrire_dataframe(df) :
    df.describe(include='all')
    return 

def aggregation_dataframe(df): 
    df['distance'].sum()
    df['air_time'].sum()
    df.groupby('origin_city_name')['air_time'].sum()
    return

def filtering_dataframe(df): 
    df[(df['origin_city_name'] == 'Abilene, TX') & (df['year'] == 2024)] 

In [ ]:
# Double boucle pour mesurer le temps, pour chaque Dataframe et pour chaque fonction d'actions agissant sur le Dataframe

for name, df in dataframes.items():
    for functions in [decrire_dataframe,aggregation_dataframe,filtering_dataframe] : 
        print(f"{name} - {mesure_temps_tache_dataframe(df_csv, functions)}")
    print("\n")

df_csv - temps passé sur la fonction decrire_dataframe: 22.18 s
df_csv - temps passé sur la fonction aggregation_dataframe: 1.19 s
df_csv - temps passé sur la fonction filtering_dataframe: 0.47 s


df_snappy - temps passé sur la fonction decrire_dataframe: 13.90 s
df_snappy - temps passé sur la fonction aggregation_dataframe: 0.52 s
df_snappy - temps passé sur la fonction filtering_dataframe: 0.33 s


df_zstd - temps passé sur la fonction decrire_dataframe: 12.92 s
df_zstd - temps passé sur la fonction aggregation_dataframe: 0.65 s
df_zstd - temps passé sur la fonction filtering_dataframe: 0.46 s


df_gzip - temps passé sur la fonction decrire_dataframe: 14.22 s
df_gzip - temps passé sur la fonction aggregation_dataframe: 0.63 s
df_gzip - temps passé sur la fonction filtering_dataframe: 0.44 s




In [ ]:
# Maintenons importons polars qui est une seconde librairie, plus récente que pandas, qui apportent de nombreuses améliorations (multithreading, ...) permettant d'aller beaucoup plus vite, sur les mêmes Dataframes que ci-dessus.

import polars as pl

df_pyarrow = pl.read_parquet("parquet_native_compression.parquet")

In [ ]:
# Nous adaptons les trois fonctions d'actions car polars n'a pas tout à fait les mêmes méthodes et ces méthodes la même syntaxe que pandas

def decrire_dataframe_polar(df) :
    df.describe()
    return 

def aggregation_dataframe_polar(df): 
    df['distance'].sum()
    df['air_time'].sum()
    df.group_by('origin_city_name').agg(pl.col('air_time').sum())
    return

def filtering_dataframe_polar(df): 
    df.filter(
    (pl.col("origin_city_name") == "Abilene, TX") & (pl.col("year") == 2024)
)

In [21]:
for functions in [decrire_dataframe_polar,aggregation_dataframe_polar,filtering_dataframe_polar] : 
    print(f"df_pyarrow - {mesure_temps_tache_dataframe(df_pyarrow, functions)}")
print("\n")

df_pyarrow - temps passé sur la fonction decrire_dataframe_polar: 1.06 s
df_pyarrow - temps passé sur la fonction aggregation_dataframe_polar: 1.26 s
df_pyarrow - temps passé sur la fonction filtering_dataframe_polar: 0.01 s




Nous pouvons observer une diminution significative du temps d'exécution.

In [23]:
df_polar_csv= pl.read_csv(r"C:\Users\Mourad\Desktop\Autoentreprise\EPSI - 2025 2026\Cours\TD1\flight_data_2024.csv")

In [24]:
for functions in [decrire_dataframe_polar,aggregation_dataframe_polar,filtering_dataframe_polar] : 
    print(f"df_pyarrow - {mesure_temps_tache_dataframe(df_polar_csv, functions)}")
print("\n")

df_pyarrow - temps passé sur la fonction decrire_dataframe_polar: 1.07 s
df_pyarrow - temps passé sur la fonction aggregation_dataframe_polar: 1.75 s
df_pyarrow - temps passé sur la fonction filtering_dataframe_polar: 0.04 s




Nous observons que même avec un fichier csv en entrée, polars est très rapide.

### Exercice 2

In [ ]:
# faker est une librairie permettant de générer des données "Dummy", toutefois réalistes.

import faker
from faker import Faker


In [ ]:
# Instanciation en langue française, afin de pouvoir générer des valeurs cohérentes pour la langue française (nom de ville, nom, prénom, métier ...)

fake  = Faker('fr_FR')

# exemple avec 10 métiers
for _ in range(10):
    print(fake.job())

ergothérapeute
pharmacien dans l'industrie
conseiller en génétique
chargé de pharmacovigilance
agent immobilier immobilière
agent arboricole
chargé de communication interne
animateur de radio et de télévision
enquêteur privé privée
médecin humanitaire


In [5]:
import pandas as pd
df_client_data = pd.read_parquet('clients_data.parquet')
df_client_data

,id_client,nom,prénom,email,téléphone,montant_achat,ville_résidence
0,1,Bonnet,Claire,claire.bonnet@exemple.fr,0622476996,686.54,Nice
1,2,Lambert,Sophie,sophie.lambert@exemple.fr,0738676681,312.73,Paris
2,3,Garcia,Antoine,antoine.garcia@exemple.fr,0768894719,349.06,Strasbourg
3,4,Lambert,Michel,michel.lambert@exemple.fr,0790341958,783.90,Bordeaux
4,5,Durand,Jean,jean.durand@exemple.fr,0785356070,646.70,Montpellier
...,...,...,...,...,...,...,...
995,996,Andre,Nathalie,nathalieandreexemple.fr,06359288449,438.79,Parix
996,997,Andre,Jean,jeanandreexemple.fr,06900176446,584.13,Parix
997,998,Roux,Sophie,sophierouxexemple.fr,06899472454,294.26,Nicz
998,999,Andre,Pierre,pierreandreexemple.fr,074030909,48.88,Nantez


##### Masquage

In [ ]:
# nous créons une nouvelle colonne qui prendra des valeurs de nom de famille générées par Faker 
df_client_data['nom_fake'] = df_client_data['nom'].apply(lambda x: fake.last_name())

# nous créons une nouvelle colonne qui prendra des valeurs de prénom générées par Faker 
df_client_data['prenom_fake'] = df_client_data['prénom'].apply(lambda x: fake.first_name())

# nous masquons les numéros de téléphone en remplaçant les 8 caractères du milieu par une asterix
df_client_data['telephone_fake'] = df_client_data['téléphone'].apply(lambda x: x[:1] +'********' +  x[-1:] )

In [7]:
df_client_data

,id_client,nom,prénom,email,téléphone,montant_achat,ville_résidence,nom_fake,prenom_fake,telephone_fake
0,1,Bonnet,Claire,claire.bonnet@exemple.fr,0622476996,686.54,Nice,Dupuy,Madeleine,0********6
1,2,Lambert,Sophie,sophie.lambert@exemple.fr,0738676681,312.73,Paris,Noël,Françoise,0********1
2,3,Garcia,Antoine,antoine.garcia@exemple.fr,0768894719,349.06,Strasbourg,Alexandre,Valentine,0********9
3,4,Lambert,Michel,michel.lambert@exemple.fr,0790341958,783.90,Bordeaux,Thierry,Renée,0********8
4,5,Durand,Jean,jean.durand@exemple.fr,0785356070,646.70,Montpellier,Rousseau,Nicolas,0********0
...,...,...,...,...,...,...,...,...,...,...
995,996,Andre,Nathalie,nathalieandreexemple.fr,06359288449,438.79,Parix,Julien,Guillaume,0********9
996,997,Andre,Jean,jeanandreexemple.fr,06900176446,584.13,Parix,Blanchet,Amélie,0********6
997,998,Roux,Sophie,sophierouxexemple.fr,06899472454,294.26,Nicz,Roussel,Gilbert,0********4
998,999,Andre,Pierre,pierreandreexemple.fr,074030909,48.88,Nantez,Seguin,Valentine,0********9


##### Anonymisation

In [ ]:

# pour l'anonymisation, comme nous généralisons le nom de ville,  j'ai téléchargé sur le site de datagouv un dataset faisant le lien entre ville et région notamment
# voici le lien : https://www.data.gouv.fr/datasets/communes-et-villes-de-france-en-csv-excel-json-parquet-et-feather/
df_siren = pd.read_csv('communes-france-2025.csv')
df_siren = df_siren[["nom_standard", "reg_nom"]]
df_siren.head()

C:\Users\Mourad\AppData\Local\Temp\ipykernel_13420\3109843058.py:1: DtypeWarning: Columns (1,12,14,16,23,24) have mixed types. Specify dtype option on import or set low_memory=False.
  df_siren = pd.read_csv('communes-france-2025.csv')


,nom_standard,reg_nom
0,L'Abergement-Clémenciat,Auvergne-Rhône-Alpes
1,L'Abergement-de-Varey,Auvergne-Rhône-Alpes
2,Ambérieu-en-Bugey,Auvergne-Rhône-Alpes
3,Ambérieux-en-Dombes,Auvergne-Rhône-Alpes
4,Ambléon,Auvergne-Rhône-Alpes


In [ ]:

# nous faisons une jointure afin d'ajouter au dataset initial le nom de la région correspondante
merged = pd.merge(
    left=df_client_data,
    right=df_siren,
    how="left",  
    left_on="ville_résidence",    
    right_on="nom_standard"     
)

merged

,id_client,nom,prénom,email,téléphone,montant_achat,ville_résidence,nom_fake,prenom_fake,telephone_fake,email_chiffrage_aes,nom_standard,reg_nom
0,1,Bonnet,Claire,claire.bonnet@exemple.fr,0622476996,686.54,Nice,Dupuy,Madeleine,0********6,gAAAAABpAqCqnVhZViDnQuiGjMcOOlJFFpBOcouvyhaLXr...,Nice,Provence-Alpes-Côte d'Azur
1,2,Lambert,Sophie,sophie.lambert@exemple.fr,0738676681,312.73,Paris,Noël,Françoise,0********1,gAAAAABpAqCqONhazc-FBHAvk8g9m04xPUj98FDMEI8Ak-...,Paris,Île-de-France
2,3,Garcia,Antoine,antoine.garcia@exemple.fr,0768894719,349.06,Strasbourg,Alexandre,Valentine,0********9,gAAAAABpAqCqhhTLs_oBDYMMMPJEZaWTjHgYRZMFSf2wAH...,Strasbourg,Grand Est
3,4,Lambert,Michel,michel.lambert@exemple.fr,0790341958,783.90,Bordeaux,Thierry,Renée,0********8,gAAAAABpAqCqMTvqbJG-5_TLGHgauiRa6S5ZURPbqM8A06...,Bordeaux,Nouvelle-Aquitaine
4,5,Durand,Jean,jean.durand@exemple.fr,0785356070,646.70,Montpellier,Rousseau,Nicolas,0********0,gAAAAABpAqCqt-Je9cYfBGmAE_YvG6WhQPoBwJeM9j-awM...,Montpellier,Occitanie
...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,996,Andre,Nathalie,nathalieandreexemple.fr,06359288449,438.79,Parix,Julien,Guillaume,0********9,gAAAAABpAqCq9kZv26cJT83MPmNjE4hUnWb9lmlXHutrDQ...,NaN,NaN
996,997,Andre,Jean,jeanandreexemple.fr,06900176446,584.13,Parix,Blanchet,Amélie,0********6,gAAAAABpAqCqN8JF4XMxOMy3eJhA6Qb8AiaDiC51LP2yxY...,NaN,NaN
997,998,Roux,Sophie,sophierouxexemple.fr,06899472454,294.26,Nicz,Roussel,Gilbert,0********4,gAAAAABpAqCqsJWqYocSHQGZIuZb-fqYuBdWkv52ZckoEU...,NaN,NaN
998,999,Andre,Pierre,pierreandreexemple.fr,074030909,48.88,Nantez,Seguin,Valentine,0********9,gAAAAABpAqCq4jEyumasjPFT3IohrEHnHXJgNgBXpljzTT...,NaN,NaN


#### 3.	Pseudonymisation/Chiffrement (Encryption Simulation

In [ ]:
# hashlib est une librairie permettant de hasher de manière idempotente (même entrée = même sortie)

import hashlib

# id_client_pseudo est une méthode où l'on applique une opération à id_client afin d'obtenir un résultat dépendant de cet id
merged['id_client_pseudo'] = merged['id_client'].apply(lambda x: (x *2 +1) * 10)

# nous hashons avec la méthode SHA256 la valeur de id_client
merged['id_client_hash'] = merged['id_client'].apply(lambda x: hashlib.sha256(str(x).encode()).hexdigest())

merged

,id_client,nom,prénom,email,téléphone,montant_achat,ville_résidence,nom_fake,prenom_fake,telephone_fake,email_chiffrage_aes,nom_standard,reg_nom,id_client_pseudo,id_client_hash
0,1,Bonnet,Claire,claire.bonnet@exemple.fr,0622476996,686.54,Nice,Dupuy,Madeleine,0********6,gAAAAABpAqCqnVhZViDnQuiGjMcOOlJFFpBOcouvyhaLXr...,Nice,Provence-Alpes-Côte d'Azur,30,6b86b273ff34fce19d6b804eff5a3f5747ada4eaa22f1d...
1,2,Lambert,Sophie,sophie.lambert@exemple.fr,0738676681,312.73,Paris,Noël,Françoise,0********1,gAAAAABpAqCqONhazc-FBHAvk8g9m04xPUj98FDMEI8Ak-...,Paris,Île-de-France,50,d4735e3a265e16eee03f59718b9b5d03019c07d8b6c51f...
2,3,Garcia,Antoine,antoine.garcia@exemple.fr,0768894719,349.06,Strasbourg,Alexandre,Valentine,0********9,gAAAAABpAqCqhhTLs_oBDYMMMPJEZaWTjHgYRZMFSf2wAH...,Strasbourg,Grand Est,70,4e07408562bedb8b60ce05c1decfe3ad16b72230967de0...
3,4,Lambert,Michel,michel.lambert@exemple.fr,0790341958,783.90,Bordeaux,Thierry,Renée,0********8,gAAAAABpAqCqMTvqbJG-5_TLGHgauiRa6S5ZURPbqM8A06...,Bordeaux,Nouvelle-Aquitaine,90,4b227777d4dd1fc61c6f884f48641d02b4d121d3fd328c...
4,5,Durand,Jean,jean.durand@exemple.fr,0785356070,646.70,Montpellier,Rousseau,Nicolas,0********0,gAAAAABpAqCqt-Je9cYfBGmAE_YvG6WhQPoBwJeM9j-awM...,Montpellier,Occitanie,110,ef2d127de37b942baad06145e54b0c619a1f22327b2ebb...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,996,Andre,Nathalie,nathalieandreexemple.fr,06359288449,438.79,Parix,Julien,Guillaume,0********9,gAAAAABpAqCq9kZv26cJT83MPmNjE4hUnWb9lmlXHutrDQ...,NaN,NaN,19930,3292bef42975c0ab63a2e9ab72143d6e2658dbd6e81a28...
996,997,Andre,Jean,jeanandreexemple.fr,06900176446,584.13,Parix,Blanchet,Amélie,0********6,gAAAAABpAqCqN8JF4XMxOMy3eJhA6Qb8AiaDiC51LP2yxY...,NaN,NaN,19950,864995ea35b82212a9a2d456a3f89833f24651c4e5ebc2...
997,998,Roux,Sophie,sophierouxexemple.fr,06899472454,294.26,Nicz,Roussel,Gilbert,0********4,gAAAAABpAqCqsJWqYocSHQGZIuZb-fqYuBdWkv52ZckoEU...,NaN,NaN,19970,462c39f8e9bbf461369150222f7493055e67079106a1a7...
998,999,Andre,Pierre,pierreandreexemple.fr,074030909,48.88,Nantez,Seguin,Valentine,0********9,gAAAAABpAqCq4jEyumasjPFT3IohrEHnHXJgNgBXpljzTT...,NaN,NaN,19990,83cf8b609de60036a8277bd0e96135751bbc07eb234256...


In [ ]:
# Chiffrement AES : chiffrement symétrique --> idéal pour chiffrer des colonnes sensibles dans un DataFrame stocké localement ou dans une base.

from cryptography.fernet import Fernet

# Génération de clé
key = Fernet.generate_key()
cipher = Fernet(key)
print(key) #clé à enregistrer de manière sécurisée afin de pouvoir décoder

# Chiffrement
merged["email_chiffrage_aes"] =  merged["email"].apply(lambda x: cipher.encrypt(x.encode()).decode())
merged

b'xjncczt7QKRoT2zdKzpT8-fiZTuAAB5QqFYYoNYc-Ik='


,id_client,nom,prénom,email,téléphone,montant_achat,ville_résidence,nom_fake,prenom_fake,telephone_fake,email_chiffrage_aes,nom_standard,reg_nom,id_client_pseudo,id_client_hash
0,1,Bonnet,Claire,claire.bonnet@exemple.fr,0622476996,686.54,Nice,Dupuy,Madeleine,0********6,gAAAAABpAqDX8JXd2kIWxnXWruao87ZEcoYID8mHO-wFqz...,Nice,Provence-Alpes-Côte d'Azur,30,6b86b273ff34fce19d6b804eff5a3f5747ada4eaa22f1d...
1,2,Lambert,Sophie,sophie.lambert@exemple.fr,0738676681,312.73,Paris,Noël,Françoise,0********1,gAAAAABpAqDXPc9JT1ZfLbGS-hT0Qco8D_HA5IS5-rdAl3...,Paris,Île-de-France,50,d4735e3a265e16eee03f59718b9b5d03019c07d8b6c51f...
2,3,Garcia,Antoine,antoine.garcia@exemple.fr,0768894719,349.06,Strasbourg,Alexandre,Valentine,0********9,gAAAAABpAqDXGbCYopz80fRHx_dt0_cppoKzm3YyAcDZTs...,Strasbourg,Grand Est,70,4e07408562bedb8b60ce05c1decfe3ad16b72230967de0...
3,4,Lambert,Michel,michel.lambert@exemple.fr,0790341958,783.90,Bordeaux,Thierry,Renée,0********8,gAAAAABpAqDXt5sKNbETyLCItwNT4TnWogI3BPFvDTiltq...,Bordeaux,Nouvelle-Aquitaine,90,4b227777d4dd1fc61c6f884f48641d02b4d121d3fd328c...
4,5,Durand,Jean,jean.durand@exemple.fr,0785356070,646.70,Montpellier,Rousseau,Nicolas,0********0,gAAAAABpAqDXv7dyzY8GFT2Xj-GJiY-nZLp30qasIKQs8e...,Montpellier,Occitanie,110,ef2d127de37b942baad06145e54b0c619a1f22327b2ebb...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,996,Andre,Nathalie,nathalieandreexemple.fr,06359288449,438.79,Parix,Julien,Guillaume,0********9,gAAAAABpAqDX2Qn11h4bFUdIiqnL7Y-W5ynUaSYTcGOIem...,NaN,NaN,19930,3292bef42975c0ab63a2e9ab72143d6e2658dbd6e81a28...
996,997,Andre,Jean,jeanandreexemple.fr,06900176446,584.13,Parix,Blanchet,Amélie,0********6,gAAAAABpAqDXKzu-4RYpXGDxZdBRz94muKRYUo72p9s7X7...,NaN,NaN,19950,864995ea35b82212a9a2d456a3f89833f24651c4e5ebc2...
997,998,Roux,Sophie,sophierouxexemple.fr,06899472454,294.26,Nicz,Roussel,Gilbert,0********4,gAAAAABpAqDXdHLBd39qJq2I2287aEs3iLP1bc4ADysNKB...,NaN,NaN,19970,462c39f8e9bbf461369150222f7493055e67079106a1a7...
998,999,Andre,Pierre,pierreandreexemple.fr,074030909,48.88,Nantez,Seguin,Valentine,0********9,gAAAAABpAqDXQWV7784bdHESNaEhxGvV6A2BkBVWe_4fm_...,NaN,NaN,19990,83cf8b609de60036a8277bd0e96135751bbc07eb234256...


##### Aparté : comment traiter les noms en erreurs ? (Data Quality)

In [1]:
# le fuzzy matching est une méthode permettant de score la correspondance entre deux termes, afin de pouvoir substituer l'un par l'autre, selon le niveau de correspondance attendu 

from rapidfuzz import process, fuzz


# création d'une fonction qui renvoie la valeur la plus proche de la valeur testée, parmi des valeurs contenues dans une liste
def best_match(name, reference_list, threshold=75):
    match = process.extractOne(name, reference_list, scorer=fuzz.ratio)
    if match and match[1] >= threshold:
        return match[0]  
    else:
        return None  

print(process.extractOne("Lyoz",["Marseille","Lyon","Paris"], scorer=fuzz.ratio))
print(process.extractOne("Marsei",["Marseille","Lyon","Paris"], scorer=fuzz.ratio))

('Lyon', 75.0, 1)
('Marseille', 80.0, 0)


Ici Lyoz et 75% proche de Lyon, donc dans la liste ["Marseille","Lyon","Paris"], Lyon est le plus logique.

75% provient de : Lyoz + Lyon = 8 lettres ; Lyoz et Lyon ont 3 lettres correspondantes.  2 * 3/8 = 0.75

80% provient de : Marsei + Marseille = 15 lettres ; Marsei et Marseille ont 6 lettres correspondantes. 2 * 6/15 = 0.8

In [22]:
merged["ville_résidence_corrected"] = merged['ville_résidence'].apply(lambda x: best_match(x, df_siren['nom_standard'].tolist()))

In [24]:
merged

,id_client,nom,prénom,email,téléphone,montant_achat,ville_résidence,nom_fake,prenom_fake,telephone_fake,email_chiffrage_aes,nom_standard,reg_nom,id_client_pseudo,id_client_hash,ville_résidence_corrected
0,1,Bonnet,Claire,claire.bonnet@exemple.fr,0622476996,686.54,Nice,Dupuy,Madeleine,0********6,gAAAAABpAqDX8JXd2kIWxnXWruao87ZEcoYID8mHO-wFqz...,Nice,Provence-Alpes-Côte d'Azur,30,6b86b273ff34fce19d6b804eff5a3f5747ada4eaa22f1d...,Nice
1,2,Lambert,Sophie,sophie.lambert@exemple.fr,0738676681,312.73,Paris,Noël,Françoise,0********1,gAAAAABpAqDXPc9JT1ZfLbGS-hT0Qco8D_HA5IS5-rdAl3...,Paris,Île-de-France,50,d4735e3a265e16eee03f59718b9b5d03019c07d8b6c51f...,Paris
2,3,Garcia,Antoine,antoine.garcia@exemple.fr,0768894719,349.06,Strasbourg,Alexandre,Valentine,0********9,gAAAAABpAqDXGbCYopz80fRHx_dt0_cppoKzm3YyAcDZTs...,Strasbourg,Grand Est,70,4e07408562bedb8b60ce05c1decfe3ad16b72230967de0...,Strasbourg
3,4,Lambert,Michel,michel.lambert@exemple.fr,0790341958,783.90,Bordeaux,Thierry,Renée,0********8,gAAAAABpAqDXt5sKNbETyLCItwNT4TnWogI3BPFvDTiltq...,Bordeaux,Nouvelle-Aquitaine,90,4b227777d4dd1fc61c6f884f48641d02b4d121d3fd328c...,Bordeaux
4,5,Durand,Jean,jean.durand@exemple.fr,0785356070,646.70,Montpellier,Rousseau,Nicolas,0********0,gAAAAABpAqDXv7dyzY8GFT2Xj-GJiY-nZLp30qasIKQs8e...,Montpellier,Occitanie,110,ef2d127de37b942baad06145e54b0c619a1f22327b2ebb...,Montpellier
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,996,Andre,Nathalie,nathalieandreexemple.fr,06359288449,438.79,Parix,Julien,Guillaume,0********9,gAAAAABpAqDX2Qn11h4bFUdIiqnL7Y-W5ynUaSYTcGOIem...,NaN,NaN,19930,3292bef42975c0ab63a2e9ab72143d6e2658dbd6e81a28...,Proix
996,997,Andre,Jean,jeanandreexemple.fr,06900176446,584.13,Parix,Blanchet,Amélie,0********6,gAAAAABpAqDXKzu-4RYpXGDxZdBRz94muKRYUo72p9s7X7...,NaN,NaN,19950,864995ea35b82212a9a2d456a3f89833f24651c4e5ebc2...,Proix
997,998,Roux,Sophie,sophierouxexemple.fr,06899472454,294.26,Nicz,Roussel,Gilbert,0********4,gAAAAABpAqDXdHLBd39qJq2I2287aEs3iLP1bc4ADysNKB...,NaN,NaN,19970,462c39f8e9bbf461369150222f7493055e67079106a1a7...,Nice
998,999,Andre,Pierre,pierreandreexemple.fr,074030909,48.88,Nantez,Seguin,Valentine,0********9,gAAAAABpAqDXQWV7784bdHESNaEhxGvV6A2BkBVWe_4fm_...,NaN,NaN,19990,83cf8b609de60036a8277bd0e96135751bbc07eb234256...,Nantes


In [23]:
merged[merged["nom_standard"].isna()]["ville_résidence"].unique()

array(['Nicx', 'Nantez', 'Lillx', 'Bordeauy', 'Nicz', 'Montpelliey',
       'Nantey', 'Strasbourz', 'Bordeauz', 'Toulousz', 'Pariz', 'Nantex',
       'Lillz', 'Parix', 'Strasboury', 'Nicy', 'Pariy', 'Lyox', 'Lyoz',
       'Montpelliez', 'Lyoy', 'Marseilly', 'Toulousx', 'Strasbourx',
       'Marseillx', 'Marseillz', 'Toulousy', 'Montpelliex'], dtype=object)

#### Contrôle d'Accès Basé sur les Rôles (RBAC)

In [28]:
def get_data_by_role(role, dataframe) :
    if role == "Analyste_Marketing":
        return dataframe[['id_client_pseudo','montant_achat', 'reg_nom']]
    elif role == "Support_Client_N1":
        return dataframe[['id_client_pseudo','montant_achat', 'telephone_fake']]
    elif role == "Admin_Sécurité":
        return dataframe
    else:
        return None

In [29]:
get_data_by_role("Support_Client_N1", merged)

,id_client_pseudo,montant_achat,telephone_fake
0,30,686.54,0********6
1,50,312.73,0********1
2,70,349.06,0********9
3,90,783.90,0********8
4,110,646.70,0********0
...,...,...,...
995,19930,438.79,0********9
996,19950,584.13,0********6
997,19970,294.26,0********4
998,19990,48.88,0********9


Un exemple d'implémentation :  WebApp, où des utilisateurs se connectent.
Je peux donner des rôles à chaque utilisateur, leur permettant d'effectuer des actions sur la WebApp.

In [30]:
roles = {
    "admin": {"create_user", "delete_user", "view_reports", "export_data"},
    "trainer": {"view_reports", "edit_content"},
    "viewer": {"view_reports"}
}

users = {
    "mourad": "admin",
    "alice": "trainer",
    "bob": "viewer"
}


In [31]:
def has_permission(user, action):
    role = users.get(user)
    return role and action in roles.get(role, set())

In [33]:
if has_permission("mourad", "export_data"):
    merged
else:
    raise PermissionError("Access denied")
